# Data Generation (Supervised Learning)

## Import Packages

In [1]:
import numpy as np
import scipy
from scipy.stats import norm
import pandapower as pp
import pandas as pd
import pandapower.networks
import tensorflow as tf
import csv
import torch

## Input Data

In [43]:
case = pp.networks.case30()
number_of_bus = case.bus.shape[0]
S_base = 100

### Z_base

V_base = np.zeros((case.line.shape[0],1))
for  i in range(case.line.shape[0]):
    V_base[i,0] = case.bus.loc[case.bus.index[case.bus.loc[:,'name']==case.line.loc[i,'to_bus']],'vn_kv']

Z_base = np.square(V_base)/S_base

### Data_form_1: PL QL PG 
P_Gen = np.zeros((case.bus.shape[0],1))
Load = np.zeros((case.bus.shape[0],2))


P_Gen[case.gen.loc[:,'bus']] = np.array([case.gen.loc[:,'p_mw']]).transpose()
Load[case.load.loc[:,'bus'],0] = np.array([case.load.loc[:,'p_mw']])
Load[case.load.loc[:,'bus'],1] = np.array([case.load.loc[:,'q_mvar']])

Data_form_1 = np.hstack((P_Gen,Load))/S_base  

           
### Data_form_2: R X  

R = np.multiply(np.array([case.line.loc[:,'r_ohm_per_km']]),np.array([case.line.loc[:,'length_km']])).transpose()
X = np.multiply(np.array([case.line.loc[:,'x_ohm_per_km']]),np.array([case.line.loc[:,'length_km']])).transpose()

Data_form_2 = np.hstack((np.multiply(R,1/Z_base),\
                       np.multiply(X,1/Z_base)))



In [45]:
case.line

,c_nf_per_km,df,from_bus,g_us_per_km,in_service,length_km,max_i_ka,max_loading_percent,name,parallel,r_ohm_per_km,std_type,to_bus,type,x_ohm_per_km
0,436.639076,1.0,0,0.0,True,1.0,0.555967,100.0,None,1,3.6450,None,1,ol,10.9350
1,291.092717,1.0,0,0.0,True,1.0,0.555967,100.0,None,1,9.1125,None,2,ol,34.6275
2,291.092717,1.0,1,0.0,True,1.0,0.277983,100.0,None,1,10.9350,None,3,ol,30.9825
3,0.000000,1.0,2,0.0,True,1.0,0.555967,100.0,None,1,1.8225,None,3,ol,7.2900
4,291.092717,1.0,1,0.0,True,1.0,0.555967,100.0,None,1,9.1125,None,4,ol,36.4500
5,291.092717,1.0,1,0.0,True,1.0,0.277983,100.0,None,1,10.9350,None,5,ol,32.8050
6,0.000000,1.0,3,0.0,True,1.0,0.384900,100.0,None,1,1.8225,None,5,ol,7.2900
7,145.546359,1.0,4,0.0,True,1.0,0.299367,100.0,None,1,9.1125,None,6,ol,21.8700
8,145.546359,1.0,5,0.0,True,1.0,0.555967,100.0,None,1,5.4675,None,6,ol,14.5800
9,0.000000,1.0,5,0.0,True,1.0,0.136853,100.0,None,1,1.8225,None,7,ol,7.2900


## Roulette Wheel Algorithm Using Normal-PDF to Generate Scenarios

In [8]:
number_of_data = 10000       # Including data for both Training and Testing

Scenario = np.zeros([number_of_data,Data_form.shape[0],Data_form.shape[1]])

for i in range(number_of_data):
    
    Scenario[i,:,:] = Data_form
    Scenario[i,load_bus_number,2:4] = norm.ppf(np.random.rand(load.shape[0],1),\
                                                              load,np.multiply(0.1,load))

Training_Data = Scenario[0:number_of_training_data,:,:]
Test_Data = Scenario[number_of_training_data:,:,:]

## Implement Power Flow to Get Labels

In [9]:
Label = np.zeros((number_of_data,number_of_bus,2))

for i in range(number_of_data):
    
    case.load.iloc[:,6:8] = Scenario[i,case.load.iloc[:,0],2:4]
    
    pp.runpp(case, algorithm='nr', calculate_voltage_angles='auto', init='auto',\
                  max_iteration='auto', tolerance_mva=1e-08, trafo_model='t',\
                  trafo_loading='current', enforce_q_lims=False,\
                  check_connectivity=True, voltage_depend_loads=True,\
                 consider_line_temperature=False)
    
    Label[i,:,:] = case.res_bus.iloc[:,0:2]
    
Training_Label = Label[0:number_of_training_data,:,:]
Test_Label = Label[number_of_training_data:,:,:]


## Exporting Data to CSV files

In [10]:
Final_Training_Data = pd.DataFrame(Training_Data.reshape(number_of_training_data,-1))
Final_Training_Label  = pd.DataFrame(Training_Label.reshape(number_of_training_data,-1))
Final_Training_Data.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/train_data_30bus.csv",index = False)
Final_Training_Label.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/train_label_30bus.csv",index = False)

Final_Test_Data   = pd.DataFrame(Test_Data.reshape(number_of_test_data,-1))
Final_Test_Label  = pd.DataFrame(Test_Label.reshape(number_of_test_data,-1))
Final_Test_Data.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/test_data_30bus.csv",index = False)
Final_Test_Label.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/test_label_30bus.csv",index = False)


